In [1]:
%pip install -q --no-cache-dir --force-reinstall airavata-python-sdk[notebook]
import airavata_jupyter_magic
%pip install 'transformers[torch]' streamlit matplotlib gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.2.0 requires cryptography!=40.0.0,!=40.0.1,<42,>=38.0.0, but you have cryptography 44.0.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.

Loaded airavata_jupyter_magic (2.0.12) 
(current runtime = local)

  %authenticate                      -- Authenticate to access high-performance runtimes.
  %request_runtime <rt> [args]       -- Request a runtime named <rt> with configuration <args>. Call multiple times to request multiple runtimes.
  %restart_runtime <rt>              -- Restart runtime <rt>. Run this if you install new dependencies or if the runtime hangs.
  %stop_runtime <rt>                 -- Stop runtime <rt> when no longer needed.
  %switch_runtime <rt>               -- Switch active runtime to <rt>. All subsequent executions will use this runtime.


In [2]:
%authenticate
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=NeuroData25VC1:cloud,expanse:shared,anvil:shared
%switch_runtime hpc_cpu

Output()

Authenticated.

Requesting runtime=hpc_cpu
cpuCount: 4
experimentName: CS_Agent
group: Default
libraries:
- python=3.10
- pip
memory: 4096
mounts: []
nodeCount: 1
pip:
- streamlit
- transformers
- torch
- matplotlib
- gradio
queue: cloud
remoteCluster: NeuroData25VC1
wallTime: 60

Requested runtime=hpc_cpu. state=EXECUTING
Switched to runtime=hpc_cpu.


In [9]:
import time
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Load model and tokenizer once at startup
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", trust_remote_code=True)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

def classify_text(user_input):
    start = time.time()
    result = classifier(user_input)[0]
    end = time.time()
    inference_time = end - start
    label = result['label']
    confidence = f"{result['score']:.4f}"
    inference_time_str = f"{inference_time:.2f}"
    return label, confidence, inference_time_str

# Create Gradio interface
iface = gr.Interface(
    fn=classify_text,
    inputs=gr.Textbox(lines=2, placeholder="Enter SAT text to classify..."),
    outputs=[
        gr.Textbox(label="Prediction Label"),
        gr.Textbox(label="Confidence Score"),
        gr.Textbox(label="Inference Time (seconds)")
    ],
    title="SAT Text Sentiment Classifier",
    description="Enter a SAT sentence to predict its sentiment (positive/negative) using DistilBERT fine-tuned on SST-2."
)

iface.launch(share=True)


⠸ Connecting to=hpc_cpu... status=CONNECTED


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://f9501205bfd0ea0b44.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
